In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
757,1,23,1,39.0
6544,1,19,1,38.0
3190,1,28,1,40.0
2064,0,32,1,40.0
4066,0,17,1,40.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6665 samples, validate on 741 samples
Epoch 1/10
6665/6665 [==============================] - 2s 342us/sample - loss: 2.4279 - mae: 1.0897 - mse: 2.4279 - val_loss: 1.7819 - val_mae: 1.0734 - val_mse: 1.7819
Epoch 2/10
6665/6665 [==============================] - 2s 231us/sample - loss: 1.4911 - mae: 0.9617 - mse: 1.4911 - val_loss: 1.7486 - val_mae: 1.0498 - val_mse: 1.7486
Epoch 3/10
6665/6665 [==============================] - 1s 186us/sample - loss: 1.4128 - mae: 0.9417 - mse: 1.4128 - val_loss: 1.4191 - val_mae: 0.9409 - val_mse: 1.4191
Epoch 4/10
6665/6665 [==============================] - 1s 167us/sample - loss: 1.3619 - mae: 0.9177 - mse: 1.3619 - val_loss: 1.5662 - val_mae: 0.9900 - val_mse: 1.5662
Epoch 5/10
6665/6665 [==============================] - 1s 156us/sample - loss: 1.3115 - mae: 0.9008 - mse: 1.3115 - val_loss: 1.4976 - val_mae: 0.9660 - val_mse: 1.4976
Epoch 6/10
6665/6665 [==============================] - 1s 169us/sample - loss: 1.3040 - mae: 0.8970 - 

In [12]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.1914473
Actual val:  7.68751907594

Predicted val:  6.3568707
Actual val:  6.9996768185

Predicted val:  7.6001854
Actual val:  8.08214652492

Predicted val:  7.5353475
Actual val:  8.2893810512

Predicted val:  7.927922
Actual val:  7.43839671988

Predicted val:  7.0144157
Actual val:  8.56275425608

Predicted val:  6.9861507
Actual val:  6.1508971098

Predicted val:  7.7312145
Actual val:  8.5098433132

Predicted val:  6.3747187
Actual val:  7.50012615324

Predicted val:  7.3007503
Actual val:  7.50012615324

